__Probing Language Models__

This notebook serves as a start for your NLP2 assignment on probing Language Models. This notebook will become part of the contents that you will submit at the end, so make sure to keep your code (somewhat) clean :-)

__note__: This assignment is not dependent on big fancy GPUs. I run all this stuff on my own 3 year old CPU, without any Colab hassle. So it's up to you to decide how you want to run it.

# Models

For the Transformer models you are advised to make use of the `transformers` library of Huggingface: https://github.com/huggingface/transformers
Their library is well documented, and they provide great tools to easily load in pre-trained models.

In [1]:
#
## Your code for initializing the transformer model(s)
#
# Note that transformer models use their own `tokenizer`, that should be loaded in as well.
#
from transformers import *

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
Discovered apex.normalization.FusedRMSNorm - will use it instead of LongT5LayerNorm
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Discovered apex.normalization.FusedRMSNorm - will use it instead of Pix2StructLayerNorm
Discovered apex.normalization.FusedRMSNorm - will use it instead of T5LayerNorm
loading file vocab.json from cach

In [2]:
#
## Your code for initializing the rnn model(s)
#
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/file/d/19Lp3AM4NEPycp_IBgoHfLc_V456pmUom/view?usp=sharing
# You can read more about this model in the original paper here: https://arxiv.org/pdf/1803.11138.pdf
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
from collections import defaultdict
from lstm.model import RNNModel
import torch


model_location = './state_dict.pt'  # <- point this to the location of the Gulordava .pt file
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)

It is a good idea that before you move on, you try to feed some text to your LMs; and check if everything works accordingly. 

# Data

For this assignment you will train your probes on __treebank__ corpora. A treebank is a corpus that has been *parsed*, and stored in a representation that allows the parse tree to be recovered. Next to a parse tree, treebanks also often contain information about part-of-speech tags, which is exactly what we are after now.

The treebank you will use for now is part of the Universal Dependencies project. I provide a sample of this treebank as well, so you can test your setup on that before moving on to larger amounts of data.

Make sure you accustom yourself to the format that is created by the `conllu` library that parses the treebank files before moving on. For example, make sure you understand how you can access the pos tag of a token, or how to cope with the tree structure that is formed using the `to_tree()` functionality.

In [3]:
# READ DATA
from typing import List
from conllu import parse_incr, TokenList


# If stuff like `: str` and `-> ..` seems scary, fear not! 
# These are type hints that help you to understand what kind of argument and output is expected.
def parse_corpus(filename: str) -> List[TokenList]:
    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses


ud_parses = parse_corpus('data/sample/en_ewt-ud-train.conllu')


# Generating Representations

We now have our data all set, our models are running and we are good to go!

The next step is now to create the model representations for the sentences in our corpora. Once we have generated these representations we can store them, and train additional diagnostic (/probing) classifiers on top of the representations.

There are a few things you should keep in mind here. Read these carefully, as these tips will save you a lot of time in your implementation.
1. Transformer models make use of Byte-Pair Encodings (BPE), that chunk up a piece of next in subword pieces. For example, a word such as "largely" could be chunked up into "large" and "ly". We are interested in probing linguistic information on the __word__-level. Therefore, we will follow the suggestion of Hewitt et al. (2019a, footnote 4), and create the representation of a word by averaging over the representations of its subwords. So the representation of "largely" becomes the average of that of "large" and "ly".


2. Subword chunks never overlap multiple tokens. In other words, say we have a phrase like "None of the", then the tokenizer might chunk that into "No"+"ne"+" of"+" the", but __not__ into "No"+"ne o"+"f the", as those chunks overlap multiple tokens. This is great for our setup! Otherwise it would have been quite challenging to distribute the representation of a subword over the 2 tokens it belongs to.


3. **Important**: If you closely examine the provided treebank, you will notice that some tokens are split up into multiple pieces, that each have their own POS-tag. For example, in the first sentence the word "Al-Zaman" is split into "Al", "-", and "Zaman". In such cases, the conllu `TokenList` format will add the following attribute: `('misc', OrderedDict([('SpaceAfter', 'No')]))` to these tokens. Your model's tokenizer does not need to adhere to the same tokenization. E.g., "Al-Zaman" could be split into "Al-"+"Za"+"man", making it hard to match the representations with their correct pos-tag. Therefore I recommend you to not tokenize your entire sentence at once, but to do this based on the chunking of the treebank. <br /><br />
Make sure to still incoporate the spaces in a sentence though, as these are part of the BPE of the tokenizer. That is, the tokenizer uses a different token id for `"man"`, than it does for `" man"`: the former could be part of `" woman"`=`" wo`"+`"man"`, whereas the latter would be the used in case *man* occurs at the start of a word. The tokenizer for GPT-2 adds spaces at the start of a token (represented as a `Ġ` symbol). This means that you should keep track whether the previous token had the `SpaceAfter` attribute set to `'No'`: in case it did not, you should manually prepend a `" "` ahead of the token.


4. The LSTM LM does not have the issues related to subwords, but is far more restricted in its vocabulary. Make sure you keep the above points in mind though, when creating the LSTM representations. You might want to write separate functions for the LSTM, but that is up to you.


5. The huggingface transformer models don't return the hidden state by default. To achieve this you can pass `output_hidden_states=True` to a model forward pass. The hidden states are then returned for all intermediate layers as well, the latest entry in this list corresponds to the top layer.


6. **N.B.**: Make sure that when you run a sentence through your model, you do so within a `with torch.no_grad():` block, and that you have run `model.eval()` beforehand as well (to disable dropout).


7. **N.B.**: Make sure to use a token's ``["form"]`` attribute, and not the ``["lemma"]``, as the latter will stem any relevant morphological information from the token. We don't want this, because we want to feed well-formed, grammatical sentences to our model.


I would like to stress that if you feel hindered in any way by the simple code structure that is presented here, you are free to modify it :-) Just make sure it is clear to an outsider what you're doing, some helpful comments never hurt.

In [4]:
# FETCH SENTENCE REPRESENTATIONS
from torch import Tensor
import pickle


# Should return a tensor of shape (num_tokens_in_corpus, representation_size)
# Make sure you correctly average the subword representations that belong to 1 token!

def get_lstm_representations(ud_parses, model, tokenizer):
    model.eval()

    with torch.no_grad():
        hidden = model.init_hidden(1)
        arr = []
        for sent in ud_parses:
            for token in sent:
                form = token["form"]
                if form in tokenizer:
                    arr.append(tokenizer[form])
                else:
                    arr.append(tokenizer["<unk>"])
        ids = torch.tensor(arr)
        rep = model(ids.unsqueeze(0), hidden)
        rep = rep.squeeze(0)
        return rep
    

def get_gpt_representations(ud_parses, model, tokenizer):
    model.eval()
    
    with torch.no_grad():
        all_sentence_reps = []
        for sentence in ud_parses:            
            no_of_original_tokens = len(sentence)
            inputs_ids_list = []
            
            for tok_idx in range(no_of_original_tokens):
                token = sentence[tok_idx]['form']
                inputs_ids_list.append(tokenizer(token, return_tensors="pt")['input_ids'])
                
            inputs_tensor = torch.cat(inputs_ids_list, -1)
            
            outputs = model(input_ids=inputs_tensor, output_hidden_states=True)
            final_reps = outputs.hidden_states[-1][0]
            
            combined_reps = []
            idx = 0
            
            for input_ids in inputs_ids_list:
                i_len = input_ids.size(-1)
                combined_reps.append(final_reps[idx:idx+i_len].mean(0))
                idx += i_len
        
            sentence_rep = torch.stack(combined_reps)
            
            all_sentence_reps.append(sentence_rep)
    return torch.cat(all_sentence_reps)


def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer) -> Tensor:    
    if isinstance(model, GPT2LMHeadModel):
        rep = get_gpt_representations(ud_parses, model, tokenizer)
    elif isinstance(model, RNNModel):
        rep = get_lstm_representations(ud_parses, model, tokenizer)
    else:
        print("NOT A SUPPORTED MODEL!!")
        return None
    
    return rep

To validate your activation extraction procedure I have set up the following assertion function as a sanity check. It compares your representation of the first sentence in the corpus against a pickled version of mine. 

For this I used `distilgpt2`.

In [5]:
def error_msg(model_name, gold_embs, embs, i2w):
    with open(f'{model_name}_tokens1.pickle', 'rb') as f:
        sen_tokens = pickle.load(f)
        
    diff = torch.abs(embs - gold_embs)
    max_diff = torch.max(diff)
    avg_diff = torch.mean(diff)
    
    print(f"{model_name} embeddings don't match!")
    print(f"Max diff.: {max_diff:.4f}\nMean diff. {avg_diff:.4f}")

    print("\nCheck if your tokenization matches with the original tokenization:")
    for idx in sen_tokens.squeeze():
        if isinstance(i2w, list):
            token = i2w[idx]
        else:
            token = i2w.convert_ids_to_tokens(idx.item())
        print(f"{idx:<6} {token}")


def assert_sen_reps(model, tokenizer, lstm, vocab):
    with open('distilgpt2_emb1.pickle', 'rb') as f:
        distilgpt2_emb1 = pickle.load(f)
        
    with open('lstm_emb1.pickle', 'rb') as f:
        lstm_emb1 = pickle.load(f)
       
    
    corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')[:1]
    
    own_distilgpt2_emb1 = fetch_sen_reps(corpus, model, tokenizer)
    own_lstm_emb1 = fetch_sen_reps(corpus, lstm, vocab)
    
    assert distilgpt2_emb1.shape == own_distilgpt2_emb1.shape, \
        f"Distilgpt2 shape mismatch: {distilgpt2_emb1.shape} (gold) vs. {own_distilgpt2_emb1.shape} (yours)"
    assert lstm_emb1.shape == own_lstm_emb1.shape, \
        f"LSTM shape mismatch: {lstm_emb1.shape} (gold) vs. {own_lstm_emb1.shape} (yours)"

    if not torch.allclose(distilgpt2_emb1, own_distilgpt2_emb1, rtol=1e-3, atol=1e-3):
        error_msg("distilgpt2", distilgpt2_emb1, own_distilgpt2_emb1, tokenizer)
    if not torch.allclose(lstm_emb1, own_lstm_emb1, rtol=1e-3, atol=1e-3):
        error_msg("lstm", lstm_emb1, own_lstm_emb1, list(vocab.keys()))


assert_sen_reps(model, tokenizer, lstm, vocab)

distilgpt2 embeddings don't match!
Max diff.: 57.6748
Mean diff. 0.3720

Check if your tokenization matches with the original tokenization:
2348   Al
12     -
57     Z
10546  aman
1058   Ġ:
1605   ĠAmerican
3386   Ġforces
2923   Ġkilled
19413  ĠSha
13848  ikh
26804  ĠAbdullah
435    Ġal
12     -
2025   An
72     i
11     ,
262    Ġthe
39797  Ġpreacher
379    Ġat
262    Ġthe
18575  Ġmosque
287    Ġin
262    Ġthe
3240   Ġtown
286    Ġof
1195   ĠQ
1385   aim
11     ,
1474   Ġnear
262    Ġthe
6318   ĠSyrian
4865   Ġborder
13     .


Next, we should define a function that extracts the corresponding POS labels for each activation, which we do based on the **``"upostag"``** attribute of a token (so not the ``xpostag`` attribute). These labels will be transformed to a tensor containing the label index for each item.

In [6]:
# FETCH POS LABELS

from typing import Optional, Dict, Tuple
# Should return a tensor of shape (num_tokens_in_corpus,)
# Make sure that when fetching these pos tags for your train/dev/test corpora you share the label vocabulary.
def fetch_pos_tags(ud_parses: List[TokenList], pos_vocab: Optional[Dict[str, int]] = None) -> Tuple[Tensor, Dict[str, int]]:
    pos_tags = []
    for sent in ud_parses:
        for token in sent:
            pos = token["upostag"]
            if pos_vocab is None:
                pos_vocab = {}
            if pos not in pos_vocab:
                new_id =  len(pos_vocab)
                pos_vocab[pos] = new_id
                pos_tags.append(new_id)
            elif pos in pos_vocab:
                old_id = pos_vocab[pos]
                pos_tags.append(old_id)
            else: 
                pass
    return torch.tensor(pos_tags), pos_vocab
                
                

Finally, we can combine all these methods to retrieve the representations (`fetch_sen_reps`) and the corresponding labels (`fetch_pos_tags`). If you are still debugging and testing your setup you can set the `use_sample` variable to `True`, and once everything works you can extract the full corpus by setting it to `False`.

The reason we pass the `train_vocab` to the data creation of the `dev` and `test` data is that we want to use the same label vocabulary across the different train/dev/test splits.

In [7]:
import os

# Function that combines the previous functions, and creates 2 tensors for a .conllu file: 
# 1 containing the token representations, and 1 containing the (tokenized) pos_tags.

def create_data(ud_parses, filename: str, lm, w2i, pos_vocab=None):
    ud_parses = parse_corpus(filename)
    
    sen_reps = fetch_sen_reps(ud_parses, lm, w2i)
    pos_tags, pos_vocab = fetch_pos_tags(ud_parses, pos_vocab=pos_vocab)
    
    return sen_reps, pos_tags, pos_vocab


lm = model  # or `lstm`
w2i = tokenizer  # or `vocab`
# lm = lstm
# w2i = vocab
use_sample = True

train_x, train_y, train_vocab = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-train.conllu'),
    lm, 
    w2i
)

dev_x, dev_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-dev.conllu'),
    lm, 
    w2i,
    pos_vocab=train_vocab
)

test_x, test_y, _ = create_data(
    ud_parses,
    os.path.join('data', 'sample' if use_sample else '', 'en_ewt-ud-test.conllu'),
    lm,
    w2i,
    pos_vocab=train_vocab
)

# Diagnostic Classification

We now have our models, our data, _and_ our representations all set! Hurray, well done. We can finally move onto the cool stuff, i.e. training the diagnostic classifiers (DCs).

DCs are simple in their complexity on purpose. To read more about why this is the case you could already have a look at the "Designing and Interpreting Probes with Control Tasks" by Hewitt and Liang (esp. Sec. 3.2).

A simple linear classifier will suffice for now, don't bother with adding fancy non-linearities to it.

I am personally a fan of the `skorch` library, that provides `sklearn`-like functionalities for training `torch` models, but you are free to train your dc using whatever method you prefer.

As this is an Artificial Intelligence master and you have all done ML1 + DL, I expect you to use your train/dev/test splits correctly ;-)

In [8]:
import torch
from torch import nn
import torch.nn.functional as F

# ! pip install skorch
from skorch import NeuralNetClassifier

In [9]:
# DIAGNOSTIC CLASSIFIER
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=15,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.output = nn.Linear(768, num_units)


    def forward(self, X, **kwargs):
        X = self.output(X)
        X = F.softmax(X, dim=-1)
        return X

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=2500,
    lr=0.001)

In [10]:
net.fit(train_x, train_y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.3967       0.1258        2.8583  0.0560
      2        2.8895       0.1085        3.0174  0.0344
      3        2.8069       0.1193        2.6833  0.0358
      4        2.6844       0.1193        2.6423  0.0358
      5        2.7162       0.1106        2.6760  0.0353
      6        2.5536       0.1106        3.1018  0.0349
      7        2.6291       0.1193        2.8488  0.0348
      8        2.5865       0.1258        2.7091  0.0351
      9        2.4941       0.1302        2.7429  0.0353
     10        2.5496       0.1367        2.5549  0.0350
     11        2.4072       0.1345        2.7042  0.0346
     12        2.4917       0.1562        2.5056  0.0354
     13        2.3709       0.1735        2.4423  0.0351
     14        2.3441       0.1779        2.4264  0.0352
     15        2.3228       0.1800        2.4268  0.0356
     16        2.3034       0.1

    107        1.4362       0.4382        1.7520  0.0361
    108        1.4313       0.4403        1.7460  0.0354
    109        1.4265       0.4469        1.7401  0.0355
    110        1.4217       0.4469        1.7343  0.0362
    111        1.4169       0.4490        1.7285  0.0347
    112        1.4122       0.4512        1.7227  0.0351
    113        1.4075       0.4555        1.7170  0.0346
    114        1.4028       0.4599        1.7113  0.0346
    115        1.3982       0.4599        1.7057  0.0349
    116        1.3936       0.4599        1.7001  0.0346
    117        1.3891       0.4599        1.6946  0.0344
    118        1.3846       0.4642        1.6891  0.0348
    119        1.3801       0.4664        1.6836  0.0346
    120        1.3757       0.4664        1.6782  0.0352
    121        1.3713       0.4664        1.6728  0.0353
    122        1.3669       0.4685        1.6675  0.0356
    123        1.3626       0.4685        1.6622  0.0361
    124        1.3583       0.4

    211        1.0823       0.5727        1.3186  0.0361
    212        1.0800       0.5727        1.3158  0.0354
    213        1.0777       0.5727        1.3130  0.0355
    214        1.0754       0.5727        1.3103  0.0359
    215        1.0731       0.5748        1.3076  0.0362
    216        1.0709       0.5748        1.3049  0.0355
    217        1.0686       0.5770        1.3022  0.0359
    218        1.0664       0.5770        1.2996  0.0362
    219        1.0642       0.5792        1.2969  0.0363
    220        1.0620       0.5792        1.2943  0.0363
    221        1.0598       0.5792        1.2917  0.0358
    222        1.0576       0.5813        1.2892  0.0356
    223        1.0555       0.5813        1.2866  0.0353
    224        1.0533       0.5835        1.2841  0.0363
    225        1.0512       0.5835        1.2815  0.0358
    226        1.0491       0.5835        1.2790  0.0359
    227        1.0470       0.5835        1.2766  0.0359
    228        1.0449       0.5

    317        0.8970       0.6529        1.1083  0.0347
    318        0.8957       0.6529        1.1068  0.0344
    319        0.8944       0.6551        1.1054  0.0357
    320        0.8930       0.6551        1.1040  0.0369
    321        0.8917       0.6573        1.1026  0.0368
    322        0.8904       0.6573        1.1012  0.0369
    323        0.8891       0.6573        1.0999  0.0363
    324        0.8878       0.6573        1.0985  0.0365
    325        0.8865       0.6594        1.0971  0.0358
    326        0.8852       0.6594        1.0957  0.0356
    327        0.8840       0.6616        1.0944  0.0349
    328        0.8827       0.6638        1.0930  0.0352
    329        0.8814       0.6681        1.0917  0.0355
    330        0.8802       0.6703        1.0903  0.0350
    331        0.8789       0.6703        1.0890  0.0344
    332        0.8777       0.6703        1.0877  0.0347
    333        0.8764       0.6703        1.0864  0.0350
    334        0.8752       0.6

    424        0.7808       0.7050        0.9877  0.0357
    425        0.7799       0.7050        0.9868  0.0374
    426        0.7790       0.7050        0.9859  0.0350
    427        0.7782       0.7050        0.9850  0.0352
    428        0.7773       0.7050        0.9842  0.0345
    429        0.7764       0.7050        0.9833  0.0347
    430        0.7755       0.7050        0.9824  0.0350
    431        0.7747       0.7050        0.9815  0.0347
    432        0.7738       0.7050        0.9807  0.0343
    433        0.7729       0.7050        0.9798  0.0353
    434        0.7721       0.7072        0.9789  0.0362
    435        0.7712       0.7072        0.9781  0.0365
    436        0.7704       0.7072        0.9772  0.0362
    437        0.7695       0.7072        0.9763  0.0354
    438        0.7687       0.7072        0.9755  0.0361
    439        0.7678       0.7072        0.9746  0.0432
    440        0.7670       0.7072        0.9738  0.0394
    441        0.7661       0.7

...

   2064        0.3540       0.8178        0.5894  0.0372
   2065        0.3540       0.8178        0.5893  0.0383
   2066        0.3539       0.8178        0.5892  0.0382
   2067        0.3538       0.8178        0.5891  0.0384
   2068        0.3537       0.8178        0.5890  0.0400
   2069        0.3536       0.8178        0.5890  0.0382
   2070        0.3535       0.8178        0.5889  0.0380
   2071        0.3534       0.8178        0.5888  0.0383
   2072        0.3533       0.8178        0.5887  0.0385
   2073        0.3532       0.8178        0.5887  0.0380
   2074        0.3531       0.8178        0.5886  0.0382
   2075        0.3531       0.8178        0.5885  0.0381
   2076        0.3530       0.8178        0.5884  0.0382
   2077        0.3529       0.8178        0.5883  0.0380
   2078        0.3528       0.8178        0.5883  0.0375
   2079        0.3527       0.8178        0.5882  0.0376
   2080        0.3526       0.8178        0.5881  0.0386
   2081        0.3525       0.8

   2174        0.3445       0.8178        0.5811  0.0373
   2175        0.3444       0.8178        0.5810  0.0374
   2176        0.3443       0.8178        0.5809  0.0382
   2177        0.3442       0.8178        0.5808  0.0379
   2178        0.3441       0.8178        0.5808  0.0381
   2179        0.3441       0.8178        0.5807  0.0380
   2180        0.3440       0.8178        0.5806  0.0369
   2181        0.3439       0.8178        0.5805  0.0368
   2182        0.3438       0.8178        0.5805  0.0386
   2183        0.3437       0.8178        0.5804  0.0381
   2184        0.3436       0.8178        0.5803  0.0380
   2185        0.3436       0.8178        0.5803  0.0381
   2186        0.3435       0.8178        0.5802  0.0370
   2187        0.3434       0.8178        0.5801  0.0368
   2188        0.3433       0.8178        0.5800  0.0370
   2189        0.3432       0.8178        0.5800  0.0364
   2190        0.3431       0.8178        0.5799  0.0365
   2191        0.3431       0.8

   2284        0.3356       0.8200        0.5734  0.0376
   2285        0.3355       0.8200        0.5733  0.0392
   2286        0.3354       0.8200        0.5732  0.0385
   2287        0.3353       0.8200        0.5732  0.0378
   2288        0.3353       0.8200        0.5731  0.0378
   2289        0.3352       0.8200        0.5730  0.0380
   2290        0.3351       0.8200        0.5730  0.0379
   2291        0.3350       0.8200        0.5729  0.0387
   2292        0.3349       0.8200        0.5728  0.0386
   2293        0.3349       0.8200        0.5728  0.0384
   2294        0.3348       0.8200        0.5727  0.0387
   2295        0.3347       0.8200        0.5726  0.0370
   2296        0.3346       0.8200        0.5726  0.0372
   2297        0.3346       0.8200        0.5725  0.0377
   2298        0.3345       0.8200        0.5724  0.0378
   2299        0.3344       0.8200        0.5724  0.0375
   2300        0.3343       0.8200        0.5723  0.0379
   2301        0.3342       0.8

   2394        0.3273       0.8221        0.5662  0.0380
   2395        0.3272       0.8221        0.5662  0.0371
   2396        0.3271       0.8221        0.5661  0.0372
   2397        0.3270       0.8221        0.5660  0.0375
   2398        0.3270       0.8221        0.5660  0.0368
   2399        0.3269       0.8221        0.5659  0.0372
   2400        0.3268       0.8221        0.5659  0.0377
   2401        0.3267       0.8221        0.5658  0.0378
   2402        0.3267       0.8221        0.5657  0.0383
   2403        0.3266       0.8221        0.5657  0.0376
   2404        0.3265       0.8221        0.5656  0.0371
   2405        0.3265       0.8221        0.5655  0.0383
   2406        0.3264       0.8221        0.5655  0.0383
   2407        0.3263       0.8221        0.5654  0.0389
   2408        0.3262       0.8221        0.5654  0.0400
   2409        0.3262       0.8221        0.5653  0.0388
   2410        0.3261       0.8221        0.5652  0.0381
   2411        0.3260       0.8

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (output): Linear(in_features=768, out_features=15, bias=True)
  ),
)

In [11]:
net.score(dev_x,dev_y)

0.807032590051458

In [12]:
net.score(test_x, test_y)

0.8169991326973114

# Trees

For our gold labels, we need to recover the node distances from our parse tree. For this we will use the functionality provided by `ete3`, that allows us to compute that directly. I have provided code that transforms a `TokenTree` to a `Tree` in `ete3` format.

In [13]:
# In case you want to transform your conllu tree to an nltk.Tree, for better visualisation

def rec_tokentree_to_nltk(tokentree):
    token = tokentree.token["form"]
    tree_str = f"({token} {' '.join(rec_tokentree_to_nltk(t) for t in tokentree.children)})"

    return tree_str


def tokentree_to_nltk(tokentree):
    from nltk import Tree as NLTKTree

    tree_str = rec_tokentree_to_nltk(tokentree)

    return NLTKTree.fromstring(tree_str)

In [14]:
# !pip install ete3
from ete3 import Tree as EteTree


class FancyTree(EteTree):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, format=1, **kwargs)
        
    def __str__(self):
        return self.get_ascii(show_internal=True)
    
    def __repr__(self):
        return str(self)


def rec_tokentree_to_ete(tokentree):
    idx = str(tokentree.token["id"])
    children = tokentree.children
    if children:
        return f"({','.join(rec_tokentree_to_ete(t) for t in children)}){idx}"
    else:
        return idx
    
def tokentree_to_ete(tokentree):
    newick_str = rec_tokentree_to_ete(tokentree)

    return FancyTree(f"{newick_str};")

In [15]:
# Let's check if it works!
# We can read in a corpus using the code that was already provided, and convert it to an ete3 Tree.

def parse_corpus(filename):
    from conllu import parse_incr

    data_file = open(filename, encoding="utf-8")

    ud_parses = list(parse_incr(data_file))
    
    return ud_parses

corpus = parse_corpus('data/sample/en_ewt-ud-train.conllu')
item = corpus[0]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree)


   /-2
  |
  |--3
  |
  |--4
  |
  |   /6 /-5
  |  |
  |  |   /-9
  |  |  |
  |  |  |--10
  |  |  |
  |  |  |--11
  |  |-8|
  |  |  |--12
  |-7|  |
  |  |  |--13
  |  |  |
  |  |   \15/-14
-1|  |
  |  |   /-16
  |  |  |
  |  |  |--17
  |  |  |
  |   \18   /-19
  |     |  |
  |     |  |--20
  |     |  |
  |     |  |-23/-22
  |      \21
  |        |--24
  |        |
  |        |   /-25
  |        |  |
  |         \28--26
  |           |
  |            \-27
  |
   \-29


As you can see we label a token by its token id (converted to a string). Based on these id's we are going to retrieve the node distances.

To create the true distances of a parse tree in our treebank, we are going to use the `.get_distance` method that is provided by `ete3`: http://etetoolkit.org/docs/latest/tutorial/tutorial_trees.html#working-with-branch-distances

We will store all these distances in a `torch.Tensor`.

Please fill in the gap in the following method. I recommend you to have a good look at Hewitt's blog post  about these node distances.

In [16]:
def create_gold_distances(corpus):
    all_distances = []

    for item in (corpus):
        tokentree = item.to_tree()
        ete_tree = tokentree_to_ete(tokentree)
 
        sen_len = len(ete_tree.search_nodes())
        distances = torch.zeros((sen_len, sen_len))
        for src_node in range(1, sen_len+1):
            for target_node in range(1, sen_len+1):
                if distances[src_node-1][target_node-1] == 0:
                     s = ete_tree&str(src_node)
                     t  = ete_tree&str(target_node)
                     dist = s.get_distance(t)
                     distances[src_node-1][target_node-1] = dist
                     distances[target_node-1][src_node-1] = dist

        # Your code for computing all the distances comes here.

        all_distances.append(distances)

    return all_distances

The next step is now to do the previous step the other way around. After all, we are mainly interested in predicting the node distances of a sentence, in order to recreate the corresponding parse tree.

Hewitt et al. reconstruct a parse tree based on a _minimum spanning tree_ (MST, https://en.wikipedia.org/wiki/Minimum_spanning_tree). Fortunately for us, we can simply import a method from `scipy` that retrieves this MST.

In [17]:
from scipy.sparse.csgraph import minimum_spanning_tree
import torch


def create_mst(distances):
    distances = torch.triu(distances).detach().numpy()
    mst = minimum_spanning_tree(distances).toarray()
    mst[mst>0] = 1.
    return mst

Let's have a look at what this looks like, by looking at a relatively short sentence in the sample corpus.

If your addition to the `create_gold_distances` method has been correct, you should be able to run the following snippet. This then shows you the original parse tree, the distances between the nodes, and the MST that is retrieved from these distances. Can you spot the edges in the MST matrix that correspond to the edges in the parse tree?

In [18]:
item = corpus[5]
tokentree = item.to_tree()
ete3_tree = tokentree_to_ete(tokentree)
print(ete3_tree, '\n')

gold_distance = create_gold_distances(corpus[5:6])[0]

mst = create_mst(gold_distance)


   /2 /-1
  |
  |--3
  |
  |--4
  |
  |   /-6
  |  |
-5|  |--7
  |-8|
  |  |   /-9
  |  |  |
  |   \12--10
  |     |
  |      \-11
  |
   \-13 



Now that we are able to map edge distances back to parse trees, we can create code for our quantitative evaluation. For this we will use the Undirected Unlabeled Attachment Score (UUAS), which is expressed as:

$$\frac{\text{number of predicted edges that are an edge in the gold parse tree}}{\text{number of edges in the gold parse tree}}$$

To do this, we will need to obtain all the edges from our MST matrix. Note that, since we are using undirected trees, that an edge can be expressed in 2 ways: an edge between node $i$ and node $j$ is denoted by both `mst[i,j] = 1`, or `mst[j,i] = 1`.

You will write code that computes the UUAS score for a matrix of predicted distances, and the corresponding gold distances. I recommend you to split this up into 2 methods: 1 that retrieves the edges that are present in an MST matrix, and one general method that computes the UUAS score.

In [19]:
def edges(mst):
    edges = set()

    edges_list = []
    for i, row in enumerate(mst):
        for j, val in enumerate(mst[i]):
            if int(val) == 1 and i<j: # i < j ensures that (1,2) is added and not (2,1) undirectional case
                edges_list.append((i,j))
                
    edges = set(edges_list)
    return edges


def calc_uuas(pred_distances, gold_distances):
    num, denom = 0, 0
    uuas = 0
    
    for pred_matrix, gold_matrix in zip(pred_distances, gold_distances):
        pred_mst = create_mst(pred_matrix)
        gold_mst = create_mst(gold_matrix)
        pred_edges = edges(pred_mst)
        gold_edges = edges(gold_mst)
        edges_pg = pred_edges & gold_edges
        num += len(edges_pg)
        denom += len(gold_edges)

    uuas = num/denom
    return uuas


# Structural Probes

We now have everything in place to start doing the actual exciting stuff: training our structural probe!
    
To make life easier for you, we will simply take the `torch` code for this probe from John Hewitt's repository. This allows you to focus on the training regime from now on.

In [20]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents


I have provided a rough outline for the training regime that you can use. Note that the hyper parameters that I provide here only serve as an indication, but should be (briefly) explored by yourself.

As can be seen in Hewitt's code above, there exists functionality in the probe to deal with batched input. It is up to you to use that: a (less efficient) method can still incorporate batches by doing multiple forward passes for a batch and computing the backward pass only once for the summed losses of all these forward passes. (_I know, this is not the way to go, but in the interest of time that is allowed ;-), the purpose of the assignment is writing a good paper after all_).

In [21]:
from torch import optim
from tqdm import tqdm

'''
Similar to the `create_data` method of the previous notebook, I recommend you to use a method 
that initialises all the data of a corpus. Note that for your embeddings you can use the 
`fetch_sen_reps` method again. However, for the POS probe you concatenated all these representations into 
1 big tensor of shape (num_tokens_in_corpus, model_dim). 

The StructuralProbe expects its input to contain all the representations of 1 sentence, so I recommend you
to update your `fetch_sen_reps` method in a way that it is easy to retrieve all the representations that 
correspond to a single sentence.
''' 


def fetch_sen_reps(ud_parses: List[TokenList], model, tokenizer, concat) -> Tensor:    
    rep = []
    if isinstance(model, GPT2LMHeadModel):
        for ud_parse in ud_parses:
            rep.append(get_gpt_representations([ud_parse], model, tokenizer))
        if concat:
            rep = nn.utils.rnn.pad_sequence(rep, batch_first=True)
    elif isinstance(model, RNNModel):
        for ud_parse in ud_parses:
            rep.append(get_lstm_representations([ud_parse], model, tokenizer))
        if concat:
            rep = nn.utils.rnn.pad_sequence(rep, batch_first=True)
    else:
        print("NOT A SUPPORTED MODEL!!")
        return None
    
    return rep
    

def init_corpus(path, model, tokenizer, concat=False, cutoff=None):
    """ Initialises the data of a corpus.
    
    Parameters
    ----------
    path : str
        Path to corpus location
    concat : bool, optional
        Optional toggle to concatenate all the tensors
        returned by `fetch_sen_reps`.
    cutoff : int, optional
        Optional integer to "cutoff" the data in the corpus.
        This allows only a subset to be used, alleviating 
        memory usage.
    """
    corpus = parse_corpus(path)[:cutoff]

    embs = fetch_sen_reps(corpus, model, tokenizer, concat=concat)    
    gold_distances = create_gold_distances(corpus)

    lengths = [sent.size(0) for sent in gold_distances]
    maxlen = int(max(lengths))
    label_maxshape = [maxlen for _ in gold_distances[0].shape]
    labels = [-torch.ones(*label_maxshape) for _ in range(len(lengths))]

    for idx, gold_dist in enumerate(gold_distances):
        length = lengths[idx]
        labels[idx][:length,:length] = gold_dist
    
    labels = torch.stack(labels)

    return labels, embs, torch.Tensor(lengths)


# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, loss_function, _data):
    # YOUR CODE HERE

    probe.eval()
    y, x, sent_lens = _data
    preds = probe(x)
    loss_score, _ = loss_function(preds, y, sent_lens)
    
    preds_new, y_new = [], []
    for i, length in enumerate(sent_lens):
        length = int(length)
        preds_resized, y_resized = preds[i, :length, :length], y[i, :length, :length]
        preds_new.append(preds_resized)
        y_new.append(y_resized)
 
    uuas_score = calc_uuas(preds_new, y_new)
    return loss_score, uuas_score


# Feel free to alter the signature of this method.
def train(_data, _dev_data, _test_data, epochs):
    emb_dim = 768
    rank = 64
    lr = 1e-1
    batch_size = 15

    probe = StructuralProbe(emb_dim, rank)
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    train_y, train_x, train_sent_lens = _data

    for epoch in range(epochs):

        for i in range(0, len(corpus), batch_size):
            probe.train()
            optimizer.zero_grad()

            # YOUR CODE FOR DOING A PROBE FORWARD PASS

            _train_batch = train_x[i:i+batch_size]
            _train_labels = train_y[i:i+batch_size]
            _train_lengths = train_sent_lens[i:i+batch_size]
            _preds = probe(_train_batch)
            batch_loss, total_sents = loss_function(_preds, _train_labels, _train_lengths)
            batch_loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, loss_function, _dev_data)
        print("Dev Loss: {}, Dev uuas: {}".format(dev_loss, dev_uuas))
        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe,loss_function,_test_data)
    print("Test Loss: {}, Test uuas: {}".format(test_loss, test_uuas))
    return probe, loss_function

In [22]:
train_data_path = 'data/sample/en_ewt-ud-train.conllu'
dev_data_path = 'data/sample/en_ewt-ud-dev.conllu'
test_data_path = 'data/sample/en_ewt-ud-test.conllu'

lm = model  # `model` or `lstm`
w2i = tokenizer  # `tokenizer` or `vocab`

train_data = init_corpus(train_data_path, lm, w2i, concat=True)
dev_data = init_corpus(dev_data_path, lm, w2i, concat=True)
test_data = init_corpus(test_data_path, lm, w2i, concat=True)

In [23]:
probe, loss_fn = train(train_data, dev_data, test_data, 200)

Dev Loss: 589.7552490234375, Dev uuas: 0.17204301075268819
Dev Loss: 407.6564025878906, Dev uuas: 0.18458781362007168
Dev Loss: 318.6963195800781, Dev uuas: 0.1639784946236559
Dev Loss: 281.4398193359375, Dev uuas: 0.1603942652329749
Dev Loss: 189.85659790039062, Dev uuas: 0.15412186379928317
Dev Loss: 129.0977020263672, Dev uuas: 0.15681003584229392
Dev Loss: 83.31909942626953, Dev uuas: 0.1774193548387097
Dev Loss: 52.66570281982422, Dev uuas: 0.17652329749103943
Dev Loss: 35.878395080566406, Dev uuas: 0.17652329749103943
Dev Loss: 24.669536590576172, Dev uuas: 0.1774193548387097
Dev Loss: 20.601089477539062, Dev uuas: 0.15412186379928317
Dev Loss: 20.056547164916992, Dev uuas: 0.16577060931899643
Dev Loss: 14.486572265625, Dev uuas: 0.17025089605734767
Dev Loss: 21.6811466217041, Dev uuas: 0.13978494623655913
Dev Loss: 22.907289505004883, Dev uuas: 0.13799283154121864
Dev Loss: 14.077306747436523, Dev uuas: 0.16129032258064516
Dev Loss: 6.057729721069336, Dev uuas: 0.145161290322580

Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.5992473363876343, Dev uuas: 0.271505376344086
Dev Loss: 1.59

## LaTeX trees

For your report you might want to add some of those fancy dependency tree plots like those of Figure 2 in the Structural Probing paper. For that you can use the following code, that outputs the corresponding LaTeX markup.

**N.B.**: for the latex tikz tree the first token in a sentence has index 1 (instead of 0), so take that into account with the predicted and gold edges that you pass to the method.

In [25]:
def print_tikz(predicted_edges, gold_edges, words):
    """ Turns edge sets on word (nodes) into tikz dependency LaTeX.
    Parameters
    ----------
    predicted_edges : Set[Tuple[int, int]]
        Set (or list) of edge tuples, as predicted by your probe.
    gold_edges : Set[Tuple[int, int]]
        Set (or list) of gold edge tuples, as obtained from the treebank.
    words : List[str]
        List of strings representing the tokens in the sentence.
    """

    string = """\\begin{dependency}[hide label, edge unit distance=.5ex]
    \\begin{deptext}[column sep=0.05cm]
    """

    string += (
        "\\& ".join([x.replace("$", "\$").replace("&", "+") for x in words])
        + " \\\\\n"
    )
    string += "\\end{deptext}" + "\n"
    for i_index, j_index in gold_edges:
        string += "\\depedge[-]{{{}}}{{{}}}{{{}}}\n".format(i_index, j_index, ".")
    for i_index, j_index in predicted_edges:
        string += f"\\depedge[-,edge style={{red!60!}}, edge below]{{{i_index}}}{{{j_index}}}{{.}}\n"
    string += "\\end{dependency}\n"
    print(string)

In [29]:
probe.eval()
y, x, sent_lens = test_data
preds = probe(x)

corpus = parse_corpus(test_data_path)[0]

preds_new, y_new = [], []
for i, length in enumerate(sent_lens):
    length = int(length)
    preds_resized, y_resized = preds[i, :length, :length], y[i, :length, :length]
    preds_new.append(preds_resized)
    y_new.append(y_resized)
    break

for pred_matrix, gold_matrix in zip(preds_new, y_new):
    pred_mst = create_mst(pred_matrix)
    gold_mst = create_mst(gold_matrix)
    pred_edges = edges(pred_mst)
    gold_edges = edges(gold_mst)

words = []
for sent in ud_parses:
    for token in sent:
        words.append(token["form"])

print_tikz(pred_edges, gold_edges, words)

\begin{dependency}[hide label, edge unit distance=.5ex]
    \begin{deptext}[column sep=0.05cm]
    Al\& -\& Zaman\& :\& American\& forces\& killed\& Shaikh\& Abdullah\& al\& -\& Ani\& ,\& the\& preacher\& at\& the\& mosque\& in\& the\& town\& of\& Qaim\& ,\& near\& the\& Syrian\& border\& .\& [\& This\& killing\& of\& a\& respected\& cleric\& will\& be\& causing\& us\& trouble\& for\& years\& to\& come\& .\& ]\& DPA\& :\& Iraqi\& authorities\& announced\& that\& they\& had\& busted\& up\& 3\& terrorist\& cells\& operating\& in\& Baghdad\& .\& Two\& of\& them\& were\& being\& run\& by\& 2\& officials\& of\& the\& Ministry\& of\& the\& Interior\& !\& The\& MoI\& in\& Iraq\& is\& equivalent\& to\& the\& US\& FBI\& ,\& so\& this\& would\& be\& like\& having\& J.\& Edgar\& Hoover\& unwittingly\& employ\& at\& a\& high\& level\& members\& of\& the\& Weathermen\& bombers\& back\& in\& the\& 1960s\& .\& The\& third\& was\& being\& run\& by\& the\& head\& of\& an\& investment\& firm\& .\& You\&